<a href="https://colab.research.google.com/github/Wuwei0-1/Dodeca/blob/main/Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import gradio as gr
import hashlib
import datetime
import sqlite3
import os
import bcrypt  # NEW: For secure password hashing
from cryptography.fernet import Fernet

# --- CONFIGURATION & KEY MANAGEMENT ---
DB_NAME = "secure_vault.db"
KEY_FILE = "secret.key"

# 1. Load or Generate the Encryption Key
if not os.path.exists(KEY_FILE):
    key = Fernet.generate_key()
    with open(KEY_FILE, "wb") as kf:
        kf.write(key)
else:
    with open(KEY_FILE, "rb") as kf:
        key = kf.read()

cipher = Fernet(key)

# --- DATABASE ENGINE ---
def init_db():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    # UPDATED: Users table now holds a password hash
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (username TEXT PRIMARY KEY, password_hash BLOB, role TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS inventions
                 (id TEXT PRIMARY KEY, owner TEXT, title TEXT, filename TEXT, data BLOB, timestamp TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS access
                 (invention_id TEXT, username TEXT)''')
    conn.commit()
    conn.close()

init_db()

# --- SECURITY UTILS ---
def hash_password(password):
    """Turns a plain password into a secure hash."""
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

def verify_password(username, password):
    """Checks if the provided password matches the stored hash."""
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password_hash FROM users WHERE username=?", (username,))
    record = c.fetchone()
    conn.close()

    if not record:
        return False

    stored_hash = record[0]
    return bcrypt.checkpw(password.encode('utf-8'), stored_hash)

# --- BACKEND LOGIC ---
def create_user(username, password, role):
    if not username or not password:
        return "❌ Error: Missing username or password."

    try:
        hashed_pw = hash_password(password)
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("INSERT INTO users VALUES (?, ?, ?)", (username, hashed_pw, role))
        conn.commit()
        return f"✅ User '{username}' created as {role}."
    except sqlite3.IntegrityError:
        return "❌ Error: Username taken."
    finally:
        conn.close()

def upload_file(owner_name, password, title, file_obj):
    # 1. Authenticate
    if not verify_password(owner_name, password):
        return "🚫 AUTH FAILED: Invalid Password."

    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    # 2. Check Role
    c.execute("SELECT role FROM users WHERE username=?", (owner_name,))
    user = c.fetchone()
    if user[0] != "Inventor":
        conn.close()
        return "❌ Only Inventors can upload."

    # 3. Process File
    if file_obj is None:
        conn.close()
        return "❌ No file selected."

    with open(file_obj, "rb") as f:
        file_bytes = f.read()

    original_filename = os.path.basename(file_obj)
    encrypted_data = cipher.encrypt(file_bytes)

    unique_id = hashlib.sha256((title + owner_name + str(datetime.datetime.now())).encode()).hexdigest()[:8]

    c.execute("INSERT INTO inventions VALUES (?, ?, ?, ?, ?, ?)",
              (unique_id, owner_name, title, original_filename, encrypted_data, str(datetime.datetime.now())))

    # Auto-grant access to owner
    c.execute("INSERT INTO access VALUES (?, ?)", (unique_id, owner_name))

    conn.commit()
    conn.close()
    return f"🔒 File Encrypted & Saved!\nID: {unique_id}"

def grant_access(owner_name, password, investor_name, invention_id):
    # 1. Authenticate
    if not verify_password(owner_name, password):
        return "🚫 AUTH FAILED: Invalid Password."

    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    # 2. Verify Ownership
    c.execute("SELECT owner FROM inventions WHERE id=?", (invention_id,))
    res = c.fetchone()
    if not res or res[0] != owner_name:
        conn.close()
        return "❌ You don't own this ID."

    c.execute("INSERT INTO access VALUES (?, ?)", (invention_id, investor_name))
    conn.commit()
    conn.close()
    return f"✅ Access granted to {investor_name}"

def download_file(user_name, password, invention_id):
    # 1. Authenticate
    if not verify_password(user_name, password):
        return None, "🚫 AUTH FAILED: Invalid Password."

    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    # Check Role
    c.execute("SELECT role FROM users WHERE username=?", (user_name,))
    role = c.fetchone()[0]

    # Get File Data
    c.execute("SELECT owner, title, filename, data FROM inventions WHERE id=?", (invention_id,))
    inv = c.fetchone()
    if not inv:
        conn.close()
        return None, "❌ ID not found."

    owner, title, fname, encrypted_blob = inv

    # Trustee Logic
    if role == "Trustee":
        conn.close()
        return None, f"🔒 TRUSTEE MODE: File exists ({fname}), but content is locked."

    # Permission Logic
    c.execute("SELECT * FROM access WHERE invention_id=? AND username=?", (invention_id, user_name))
    if not c.fetchone():
        conn.close()
        return None, "🚫 ACCESS DENIED: You do not have the key."

    # Decrypt
    try:
        decrypted_bytes = cipher.decrypt(encrypted_blob)
        output_path = f"decrypted_{fname}"
        with open(output_path, "wb") as f:
            f.write(decrypted_bytes)
        conn.close()
        return output_path, "✅ Decryption Successful. Download below."
    except:
        conn.close()
        return None, "❌ Decryption Error."

def ui_audit():
    # Audit log does not require password, it is public ledger (or could be restricted)
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT timestamp, owner, title, id FROM inventions ORDER BY timestamp DESC LIMIT 10")
    data = c.fetchall()
    conn.close()
    log = "--- SYSTEM AUDIT TRAIL ---\n"
    for row in data:
        log += f"[{row[0]}] User: {row[1]} | Project: {row[2]} | ID: {row[3]}\n"
    return log

# --- THEME & UI ---
theme = gr.themes.Soft(primary_hue="emerald", secondary_hue="slate")

with gr.Blocks(theme=theme, title="SecureVault v2") as app:
    gr.Markdown("# 🛡️ Secure Invention Vault (v2)")
    gr.Markdown("### 🔒 Identity Verified Storage")

    with gr.Tab("1. Register Identity"):
        with gr.Row():
            reg_name = gr.Textbox(label="Username")
            reg_pass = gr.Textbox(label="Password", type="password") # Masked input
            reg_role = gr.Dropdown(["Inventor", "Investor", "Engineer", "Trustee"], label="Role", value="Inventor")
        reg_btn = gr.Button("Create Secure Profile", variant="primary")
        reg_out = gr.Textbox(label="Status")
        reg_btn.click(create_user, [reg_name, reg_pass, reg_role], reg_out)

    with gr.Tab("2. Upload (Inventor)"):
        with gr.Row():
            inv_user = gr.Textbox(label="Username")
            inv_pass = gr.Textbox(label="Password", type="password")
        inv_title = gr.Textbox(label="Project Title")
        inv_file = gr.File(label="Upload Prototype", type="filepath")
        up_btn = gr.Button("Encrypt & Upload", variant="primary")
        up_out = gr.Textbox(label="Result")
        up_btn.click(upload_file, [inv_user, inv_pass, inv_title, inv_file], up_out)

        gr.Markdown("### Grant Key")
        with gr.Row():
            g_owner = gr.Textbox(label="Your Username")
            g_pass = gr.Textbox(label="Your Password", type="password")
            g_inv = gr.Textbox(label="Investor Username")
            g_id = gr.Textbox(label="Invention ID")
        g_btn = gr.Button("Grant Access")
        g_out = gr.Textbox(label="Status")
        g_btn.click(grant_access, [g_owner, g_pass, g_inv, g_id], g_out)

    with gr.Tab("3. Decrypt (Viewer)"):
        with gr.Row():
            v_user = gr.Textbox(label="Username")
            v_pass = gr.Textbox(label="Password", type="password")
            v_id = gr.Textbox(label="Invention ID")
        v_btn = gr.Button("Authenticate & Decrypt", variant="primary")
        with gr.Row():
            v_status = gr.Textbox(label="Status")
            v_file = gr.File(label="Decrypted Asset")
        v_btn.click(download_file, [v_user, v_pass, v_id], [v_file, v_status])

    with gr.Tab("4. Trustee Log"):
        log_btn = gr.Button("Pull Logs")
        log_out = gr.Textbox(label="Immutable Audit Trail", lines=10)
        log_btn.click(ui_audit, None, log_out)

app.launch()

/tmp/ipython-input-1463765602.py:193: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, title="SecureVault v2") as app:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://788fe68a02d71ebe5e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
import os

if os.path.exists('secure_vault.db'):
    os.remove('secure_vault.db')
    print('secure_vault.db deleted successfully.')
else:
    print('secure_vault.db does not exist.')


secure_vault.db deleted successfully.


In [11]:
!pip install bcrypt